# Xarray/Zarr/Icechunk on S3

You will need to run this notebook in a `conda` environment created from `environment.yml`.

In [1]:

import zarr

from icechunk import IcechunkStore, Storage

In [2]:
s3_storage = Storage.s3_from_env("bucket=icechunk-test",prefix="oscar-demo-repository")

## Create a new Zarr store backed by Icechunk

This example uses a S3 store

In [ ]:
store = await IcechunkStore.create(
    storage=s3_storage,
    mode="w",
)

## Real data

In [4]:
import xarray as xr

In [5]:
import fsspec

fs = fsspec.filesystem("s3")

In [ ]:
oscar = xr.open_dataset(
    fs.open("s3://earthmover-sample-data/netcdf/oscar_vel2018.nc"),
    chunks={},
    engine="h5netcdf",
)
oscar

In [ ]:
group = zarr.group(store=store, overwrite=True)
group

In [ ]:
import time

for var in oscar:
    print(var)
    tic = time.time()
    group.create_array(
        name=var,
        shape=oscar[var].shape,
        fill_value=-1234567,
        dtype=oscar[var].dtype,
        data=oscar[var],
        exists_ok=True,
    )
    print(await store.commit(f"wrote {var}"))
    print(f"commited; {time.time() - tic} seconds")

## Open store

1. why is zarr writing anything
2. why is icehcunk letting zarr write something with mode="r"
3. whay aren't we checking out main.
4. can't read data back

In [ ]:
store = await IcechunkStore.open_existing(
    storage=s3_storage,
    mode="r",
)
store

In [ ]:
store

In [6]:
root_group = zarr.open_group(store=store)

In [ ]:
root_group.members()

In [8]:
await store.reset()

In [9]:
await store.checkout("Y9P6P8BYRXEN5JCD0XNK1PV79M")

In [ ]:
root_group.members()

In [ ]:
u = root_group["u"][0, 0, :, :]